In [1]:
import pandas as pd

In [14]:
trip = pd.read_excel(r'J:\Projects\Surveys\HHTravel\Survey2017\Data\Export\Version 1\Public\2017-pr1-5-trip.xlsx',
                       sheetname='5-Trip', skiprows=1)
#sample analysis, looks good
#trip.groupby('mode_1').count()
#trip.groupby('d_purp').count()
#trip.groupby('o_purp').count()

In [26]:
# Rename some variables for convenience
trip['o_purp'] = trip['origin_purpose']
trip['d_purp'] = trip['dest_purpose']
trip['a_dur'] = trip['activity_duration']

In [40]:
#Begin to make transformations

#In order of appearance in sample 2015 trip csv
final_trip = pd.DataFrame()

#hhno
final_trip['hhno'] = trip['hhid']

#pno
final_trip['pno'] = trip['pernum']

#day
final_trip['day'] = trip['daynum'].astype(int)

#tour
final_trip['tour'] = -1

#half
final_trip['half'] = -1

#tseg
final_trip['tseg'] = -1

#tsvid
final_trip['tsvid'] = trip['tripnum']

#opurp
#using guidelines from Daysim2.1 Users Guide
def opurp (row):
   if row['o_purp'] == 1 :
      return 0
   if row['o_purp'] == 10 or row['o_purp'] == 11 or row['o_purp'] == 14 :
      return 1
   if row['o_purp'] == 6 :
      return 2
   if row['o_purp'] == 9 :
      return 3
   if row['o_purp'] == 33 or row['o_purp'] == 54 or row['o_purp'] == 56 or row['o_purp'] == 61 :
      return 4
   if row['o_purp'] == 30 or row['o_purp'] == 32 :
      return 5
   if row['o_purp'] == 50 :
      return 6
   if row['o_purp'] == 52 or row['o_purp'] == 62:
      return 7
   if row['o_purp'] == 53 or row['o_purp'] == 51 :
      return 8
   if row['o_purp'] == 34 :
      return 9
   if row['o_purp'] == 60 :
      return 10
   return 11

final_trip['opurp'] = trip.apply (lambda row: opurp (row),axis=1)

#dpurp
def dpurp (row):
   if row['d_purp'] == 1 :
      return 0
   if row['d_purp'] == 10 or row['d_purp'] == 11 or row['d_purp'] == 14 :
      return 1
   if row['d_purp'] == 6 :
      return 2
   if row['d_purp'] == 9 :
      return 3
   if row['d_purp'] == 33 or row['d_purp'] == 54 or row['d_purp'] == 56 or row['d_purp'] == 61 :
      return 4
   if row['d_purp'] == 30 or row['d_purp'] == 32 :
      return 5
   if row['d_purp'] == 50 :
      return 6
   if row['d_purp'] == 52 or row['d_purp'] == 62:
      return 7
   if row['d_purp'] == 53 or row['d_purp'] == 51 :
      return 8
   if row['d_purp'] == 34 :
      return 9
   if row['d_purp'] == 60 :
      return 10
   return 11

final_trip['dpurp'] = trip.apply (lambda row: dpurp (row),axis=1)

#oadtyp
final_trip['oadtyp'] = -1

#dadtyp
final_trip['dadtyp'] = -1

#opcl
final_trip['opcl'] = -1

#otaz
final_trip['otaz'] = -1

#dpcl
final_trip['dpcl'] = -1

#dtaz
final_trip['dtaz'] = -1

#mode (subject to revision, particularly with transit designations)
def mode (row):
   if row['mode_1'] == 1 :
      return 1
   if row['mode_1'] == 2 :
      return 2
   if (row['mode_1'] in range(3,17) or row['mode_1'] in range(33,35)) and row['travelers_total'] == 1:
      return 3
   if (row['mode_1'] in range(3,17) or row['mode_1'] in range(33,35)) and row['travelers_total'] == 2:
      return 4
   if (row['mode_1'] in range(3,17) or row['mode_1'] in range(33,35)) and row['travelers_total'] > 2:
      return 5
   if row['mode_1'] == 23 or row['mode_1'] == 32 or row['mode_1'] == 52 or row['mode_1'] in range(26,29) or row['mode_1'] in range(41,43):
      return 6
   if row['mode_1'] == 24 :
      return 8
   return 9

final_trip['mode'] = trip.apply (lambda row: mode (row),axis=1)

#pathtype (removing network options: 1 = local bus, 2 = light rail, 3 = premium bus, 4 = commuter rail, 5 = ferry/water taxi)
def pathtype (row):
   if row['mode_1'] == 23 :
      return 1
   if row['mode_1'] == 52 :
      return 2
   if row['mode_1'] == row['mode_1'] in range(26,29) :
      return 3
   if row['mode_1'] == row['mode_1'] in range(41,43) :
      return 4 
   if row['mode_1'] == 32 :
      return 5  
   return 0

final_trip['pathtype'] = trip.apply (lambda row: pathtype (row),axis=1)

#dorp
def dorp (row):
   if row['driver'] == 1 :
      return 1
   if row['driver'] == 2 :
      return 2
   return 3
    
final_trip['dorp'] = trip.apply (lambda row: dorp (row),axis=1)
rmove_trip['dorp'] = rmove.apply (lambda row: dorp (row),axis=1)

#deptm
final_trip['deptm'] = trip['depart_time_mam']

#arrtm
final_trip['arrtm'] = trip['arrival_time_mam']

#endacttm
trip['a_dur'].fillna(0, inplace=True)
final_trip['endacttm'] = trip['a_dur'].astype(int)

#travtime (best guess)
trip['google_duration'].fillna(0, inplace=True)
trip['google_duration'].replace(' ', 0, inplace=True)
final_trip['travtime'] = trip['google_duration'].astype(int)

#travcost (best guess)
def toll (row):
   if row['toll_pay'] == 3 :
      return 1
   if row['toll_pay'] == 4 :
      return 3
   if row['toll_pay'] == 5 :
      return 5 
   if row['toll_pay'] == 6 :
      return 7  
   if row['toll_pay'] == 7 :
      return 9   
   if row['toll_pay'] == 8 :
      return 10    
   return 0

final_trip['travcost'] = trip.apply (lambda row: toll (row),axis=1)

#travdist (best guess)
final_trip['travdist'] = trip['trip_path_distance']

# Weights
final_trip['trexpfac'] = trip['trip_wt_final']

In [41]:
final_trip.to_csv(r'J:\Projects\Surveys\HHTravel\Survey2017\Data\Export\Version 1\Daysim\trip.csv', index=False)
final_trip.to_csv(r'J:\Projects\Surveys\HHTravel\SurveyComparisons\14_17_daysim_format\trip_17.csv', index=False)